In [ ]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

# Banking and Unemployment
---

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gmaps
import requests
import time
from census import Census
from us import states

# census key
from config import (census_key, gkey)

# Census API Key
c = Census(census_key, year=2013)

ModuleNotFoundError: No module named 'gmaps'

## Data Retrieval

In [2]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E", "B17001_002E"), {
                         'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)


# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Poverty Rate"]]

# Visualize
print(len(census_pd))
census_pd.head()

NameError: name 'c' is not defined

## Combine Data

In [3]:
# Import the original data we analyzed earlier. Use dtype="object" to match other
census_data_original = pd.read_csv(
    "../Resources/zip_bank_data.csv", dtype="object", encoding="utf-8")

# Visualize
census_data_original.head()

,Zipcode,Address,Median Age,Household Income,Per Capita Income,Lat,Lng,Bank Count
0,624,"Hastings, MI 49058, USA",40.8,46777,22137,42.6306916,-85.2929384,9
1,692,"Ball, LA 71405, USA",35.8,55242,23941,31.4061799,-92.396174,12
2,730,"Great Mills, MD 20634, USA",31.9,79944,35961,38.2201614,-76.4967919,9
3,757,"Williamsport, TN 38487, USA",41.6,38125,18884,35.7310368,-87.2419299,0
4,957,"Marion, ND 58466, USA",44.5,69844,36981,46.5594224,-98.3481542,1


In [4]:
# Merge the two data sets along zip code
census_data_complete = pd.merge(
    census_data_original, census_pd, how="left", on=["Zipcode", "Zipcode"])

# Remove rows missing data
census_data_complete = census_data_complete.dropna()

# Visualize
census_data_complete.head()

NameError: name 'census_pd' is not defined

## Heatmap of poverty rate

In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

NameError: name 'gmaps' is not defined

In [6]:
# Store 'Lat' and 'Lng' into  locations 
locations = census_data_complete[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
poverty_rate = census_data_complete["Poverty Rate"].astype(float)

NameError: name 'census_data_complete' is not defined

In [7]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

NameError: name 'gmaps' is not defined

In [8]:
# Convert bank rate to list
bank_rate = census_data_complete["Bank Count"].tolist()

NameError: name 'census_data_complete' is not defined

In [9]:
# Create bank symbol layer
bank_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
)


fig = gmaps.figure()
fig.add_layer(bank_layer)

fig

NameError: name 'gmaps' is not defined

In [ ]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(bank_layer)

fig